# Tools to see what happen per pair of images instead of full homography


Before start, run something like:

`python3 create_list.py correspondances --pto_filename="/mnt/c/Users/Ignasi/Downloads/FOTOS_BCN_2022-04-06/panorama_stiching/DJI_0623 - DJI_0627.pto" --name=prueba --windows`

<br>

When it works, run something like:

`python3 create_platja_models.py --model_name=kk --json_path=./data_lists/prueba/prueba.json --csv_path=./data_lists/prueba/prueba.csv --masks_path=/mnt/c/Users/Ignasi/Downloads/FOTOS_BCN_2022-04-06/panorama_stiching/masks/ --output_path=outputs/`

In [47]:

!python3 create_list.py correspondances --pto_filename="/mnt/c/Users/Ignasi/Downloads/platges_per_carpeta/2/1/DJI_0673 - DJI_0680.pto" --name=2_1 --windows


In [55]:
%matplotlib inline


from matplotlib import pyplot as plt
import numpy as np

from create_platja_models import load_data, generate_correspondances, create_homographies
from frameworks.opencv.homography.stiching import make_stiching, color_matrix_applier, color_blender
from frameworks.opencv.homography.utils import compute_dimensions


plt.rcParams['figure.figsize'] = [25, 15]
np.set_printoptions(formatter={'float' : lambda x: f"{x:10.3f}"})


In [58]:

#data_root = '/mnt/c/Users/Ignasi/Downloads/FOTOS_BCN_2022-04-06/'
data_root = '/mnt/c/Users/Ignasi/Downloads/platges_per_carpeta/2/1/'

name = '2_1'
json_path = f'./data_lists/{name}/{name}.json' # './data_lists/prueba/prueba.json'
csv_path = f'./data_lists/{name}/{name}.csv' # './data_lists/prueba/prueba.csv'

OFFSET_SUBSET = 4

#OFFSET = OFFSET_SUBSET % (len(v_img) - 2 + 1)

MAX_HEIGHT = 6000 # None
MAX_WIDTH = 8000 # None


In [59]:

dataset, correspondances_df = load_data(json_path, csv_path, data_root=data_root)

v_img = []
for d in list(dataset[0]):
    print(d[1])
    v_img.append(d[0])

OFFSET = OFFSET_SUBSET % (len(v_img) - 2 + 1)
    
correspondances_df_subset = correspondances_df[((correspondances_df['img1'] == OFFSET) | (correspondances_df['img2'] == OFFSET)) & ((correspondances_df['img1'] == OFFSET + 1) | (correspondances_df['img2'] == OFFSET + 1))]
v_img_subset = v_img[OFFSET : OFFSET + 2]
     
v_homographies = create_homographies(correspondances_df_subset, v_img_subset)

DJI_0673.JPG
DJI_0674.JPG
DJI_0675.JPG
DJI_0676.JPG
DJI_0677.JPG
DJI_0678.JPG
DJI_0679.JPG
DJI_0680.JPG


In [60]:
v_homographies

[matrix([[     1.000,      0.000,   4422.000],
         [     0.000,      1.000,   4118.000],
         [     0.000,      0.000,      1.000]]),
 matrix([[     1.738,     -0.696,   2538.607],
         [     0.044,      9.934,      0.065],
         [     0.000,      0.002,      0.348]])]

In [61]:

composed_img = np.ones(v_img_subset[0].shape[:2])
max_x, min_x, max_y, min_y = compute_dimensions(composed_img, composed_img, v_homographies[0])
width = int(max_x - min_x + 1)
height = int(max_y - min_y + 1)
composed_img = np.ones((height, width))

current_img = np.ones(v_img_subset[1].shape[:2])
max_x, min_x, max_y, min_y = compute_dimensions(composed_img, current_img, v_homographies[1])
width = int(max_x - min_x + 1)
height = int(max_y - min_y + 1)

print((width, height))
print(f'{width * height} px')
print(f'{width * height * 3 * 8 // 8:,} Bytes if each color uses 8 bits ({int(width * height * 3 * 8 / 8 / 1e6)} MB)'.replace(',', ' '))
print(f'{width * height * 3 * 32 // 8:,} Bytes if each color uses 32 bits ({int(width * height * 3 * 32 / 8 / 1e6)} MB)'.replace(',', ' '))
print(f'{width * height * 3 * 64 // 8:,} Bytes if each color uses 64 bits ({int(width * height * 3 * 64 / 8 / 1e6)} MB)'.replace(',', ' '))


(32639, 7766)
253474474 px
760 423 422 Bytes if each color uses 8 bits (760 MB)
3 041 693 688 Bytes if each color uses 32 bits (3041 MB)
6 083 387 376 Bytes if each color uses 64 bits (6083 MB)


In [ ]:

stiched = make_stiching(v_img_subset, v_homographies, matrix_applier=color_matrix_applier, blender=color_blender, max_width=MAX_WIDTH, max_heignt=MAX_HEIGHT)
stiched = stiched.astype(int)

plt.imshow(stiched)
plt.show()


In [ ]:

handpicked_kps, handpicked_descriptors = generate_correspondances(correspondances_df_subset)

fig, ax = plt.subplots(1, len(handpicked_kps))
for i, (img, kps, des) in enumerate(zip(v_img_subset[::-1], handpicked_kps[::-1], handpicked_descriptors[::-1])):

    try:
        kps_x_red, kps_y_red = list(zip(*[(kp.pt[0], kp.pt[1]) for kp, de in zip(kps, des) if de[1] == 0]))
    except:
        kps_x_red, kps_y_red = [], []
    try:
        kps_x_cyan, kps_y_cyan = list(zip(*[(kp.pt[0], kp.pt[1]) for kp, de in zip(kps, des) if de[1] == 1]))
    except:
        kps_x_cyan, kps_y_cyan = [], []
    
    print(f'Keypoints of the image #{i + 1}')
    print([kp.pt for kp in kps], end='\n---\n')
    
    ax[i].imshow(img.astype(np.uint8)) #, cmap=plt.cm.gray)
    ax[i].plot(kps_x_cyan, kps_y_cyan, color='cyan', marker='o', linestyle='None', markersize=6)
    ax[i].plot(kps_x_red, kps_y_red, color='red', marker='o', linestyle='None', markersize=6)
    ax[i].title.set_text(f'image #{i + 1}')

plt.show()

---